<a href="https://colab.research.google.com/github/cappelchi/Coronavirus/blob/master/Biopython_coronavirus_notebook_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [3]:
import os
import sys

from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
Biopython version: 1.76


In [0]:
input_file = "unknown-sequence.fa"

fasta_loc = ("https://raw.githubusercontent.com/chris-rands/"
             "biopython-coronavirus/master/unknown-sequence.fa")

if not os.path.exists(input_file):
    urlretrieve(fasta_loc, input_file)

In [5]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


In [0]:
record = SeqIO.read(input_file, "fasta")

In [7]:
record.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())

In [8]:
print("Sequence length (bp)", len(record))

Sequence length (bp) 29903


In [9]:
print("GC content (%)", GC(record.seq))

GC content (%) 37.97277865097148


## **Compare to other genome sequences**
Let's use BLAST to align the unknown sequence to other annoated sequences in the NCBI nt database, which contains sequences from many different species from accross the tree of life.

This may take ~10 minutes since we are doing an online search against many sequences (for larger queries, it would sensible to run BLAST locally instead; see Bio.Blast.Applications)

In [10]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

CPU times: user 236 ms, sys: 73 ms, total: 309 ms
Wall time: 34min 1s


In [0]:
blast_qresult = SearchIO.read(result_handle, "blast-xml")

In [12]:
print(blast_qresult)

Program: blastn (2.10.0+)
  Query: No (29903)
         definition line
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|1798174254|ref|NC_045512.2|  Severe acute respirator...
            1      1  gi|1819735426|gb|MT121215.1|  Severe acute respiratory ...
            2      1  gi|1805293633|gb|MT019531.1|  Severe acute respiratory ...
            3      1  gi|1820247323|dbj|LC529905.1|  Severe acute respiratory...
            4      1  gi|1818244627|gb|MT135044.1|  Severe acute respiratory ...
            5      1  gi|1818244605|gb|MT135042.1|  Severe acute respiratory ...
            6      1  gi|1818244594|gb|MT135041.1|  Severe acute respiratory ...
            7      1  gi|1805293611|gb|MT019529.1|  Severe acute respiratory ...
            8      1  gi|1802633808|gb|MN996528.1|  Severe acute res

In [13]:
[hit.description for hit in blast_qresult[:5]]

['Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/SH01/human/2020/CHN, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate BetaCoV/Wuhan/IPBCAMS-WH-03/2019, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 TKYE6182_2020 RNA, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/235/human/2020/CHN, complete genome']


Well that looks fairly conclusive, without doing any quantitative analyses, it's already very likely we have a coronavirus genome here, specifically SARS2-CoV-2 that was the cause of the COVID19 pandemic!

Let's have a look at the first result in a bit more detail to check some of the alignment metrics

In [0]:
first_hit = blast_qresult[0]

In [16]:
first_hit.description

'Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome'

In [17]:
first_hsp = first_hit[0]
print(first_hsp.evalue, first_hsp.bitscore)

0.0 53927.4


In [18]:
print(first_hsp.aln)

DNAAlphabet() alignment with 2 rows and 29903 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA No
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA gi|1798174254|ref|NC_045512.2|



The alignment appears of high quality and not merely a spurious hit

We could view/save the full sequence alignment, for illustration purposes, here is just the first 100 characters in FASTA format

In [19]:
print(first_hsp.aln.format("fasta")[:100])

>No definition line
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT
GTTCTCTAAACGAACTTTA



Extract annotations on the matching genome sequence

Let's extract a bit more structured meta-data on the top matching sequence homologous sequence using NCBI Entrez via Biopython to extract a GenBank file


In [20]:
NCBI_id = first_hit.id.split('|')[3]
NCBI_id

'NC_045512.2'

In [0]:
Entrez.email = "cappelchi@gmail.com"  # Always tell NCBI who you are

In [0]:
handle = Entrez.efetch(db="nucleotide", id= NCBI_id, retmode="text", rettype="gb")

In [0]:
genbank_record = SeqIO.read(handle, "genbank")

In [24]:
genbank_record

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', IUPACAmbiguousDNA()), id='NC_045512.2', name='NC_045512', description='Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', dbxrefs=['BioProject:PRJNA485481'])

In [25]:
print("Is it single or double stranded and a DNA or RNA virus?\n",
      genbank_record.annotations["molecule_type"])

Is it single or double stranded and a DNA or RNA virus?
 ss-RNA


In [26]:
print("What are the relevant references/labs who generated the data?\n")
for reference in genbank_record.annotations["references"]:
    print(reference)

What are the relevant references/labs who generated the data?

location: [13475:13503]
authors: Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F., Atkins,J.F. and Howard,M.T.
title: Programmed ribosomal frameshifting in decoding the SARS-CoV genome
journal: Virology 332 (2), 498-510 (2005)
medline id: 
pubmed id: 15680415
comment: 

location: [29727:29768]
authors: Robertson,M.P., Igel,H., Baertsch,R., Haussler,D., Ares,M. Jr. and Scott,W.G.
title: The structure of a rigorously conserved RNA element within the SARS virus genome
journal: PLoS Biol. 3 (1), e5 (2005)
medline id: 
pubmed id: 15630477
comment: 

location: [29608:29657]
authors: Williams,G.D., Chang,R.Y. and Brian,D.A.
title: A phylogenetically conserved hairpin-type 3' untranslated region pseudoknot functions in coronavirus RNA replication
journal: J. Virol. 73 (10), 8349-8355 (1999)
medline id: 
pubmed id: 10482585
comment: 

location: [0:29903]
authors: Wu,F., Zhao,S., Yu,B., Chen,Y.-M., Wang,W., Hu,Y., Song,Z.-

In [27]:
len(genbank_record.features)

57

In [28]:
{feature.type for feature in genbank_record.features}

{"3'UTR", "5'UTR", 'CDS', 'gene', 'mat_peptide', 'source', 'stem_loop'}

In [29]:
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]
len(CDSs)

12

In [30]:
CDSs[0].qualifiers["gene"]

['ORF1ab']

In [0]:
protein_seq = Seq(CDSs[0].qualifiers["translation"][0])

In [32]:
protein_seq

Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...VNN')

In [33]:
print("Does the sequence begin with a start codon?\n",
      protein_seq.startswith("M"))

Does the sequence begin with a start codon?
 True


In [34]:
print(CodonTable.unambiguous_dna_by_id[1])

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

However, we can't perform an exact "reverse translation" of course, since several amino acids are produced by the same codon. Note that if instead we started with the nucleotide sequence, then we could use Biopython's .transcribe() and .translate() functions to convert sequences from DNA to RNA and DNA to protein respectively.

In [36]:
print("Protein sequence length in amino acids", len(protein_seq))

Protein sequence length in amino acids 7096



It is a long protein for a virus. Let's check the annotation

In [37]:
CDSs[0].qualifiers["product"]

['ORF1ab polyprotein']


So it looks like this is a polyprotein, which explains why it is a relatively long protein. Polyproteins are a typical feature of some viral genomes where smaller proteins are joined together, providing a particular organization of the viral proteome.